In [22]:
import random
from datetime import datetime, date, timedelta
import pandas as pd
import itertools
import json
import numpy
import pm4py
import os
import functools

In [23]:
def getRandomTime(min_x, max_x, duration, start_time):
    if type(start_time) == type('a'):
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S:%f")
    random_duration = start_time + timedelta(minutes=duration)
    return (random_duration.strftime('%Y-%m-%d %H:%M:%S:%f'), duration)

In [24]:
def calculateDuration(df, start_time):
    for index, row in df.iterrows():
        if index != 0:
            previous = df.loc[index -1, "End_time"]
            a = getRandomTime(row["Min"],row["Max"], row["Duration"], previous)
            df.at[index, "start_time"] = previous
            df.at[index, "End_time"] = datetime.strptime(a[0], "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "Duration"] = a[1]
        else:
            a = getRandomTime(row["Min"],row["Max"], row["Duration"], start_time)
            df.at[index, "start_time"] = datetime.strptime(start_time.strftime('%Y-%m-%d %H:%M:%S:%f'), "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "End_time"] = datetime.strptime(a[0], "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "Duration"] = a[1]
    return(df)

In [25]:
# Each Activity have 3 properties: Activity, minimum time, maximum time

def nightSleep():
    return ['sleep', '00:15:00:00','04:00:00:00']

def nightSleep3B():
    return ['sleep', '00:00:00:00','01:00:00:00']

In [26]:
def goBathroomNight():
    return ['bathroom', '00:05:00:00', '00:15:00:00']
 
def goBathroomNight3B():
    return ['bathroom', '00:03:00:00', '00:10:00:00']

In [27]:
def wakeUp():
    return ['wake up', '00:01:00:00','00:15:00:00']

In [28]:
def waitingTime3A():
    return ['waiting', '00:01:00:00', '00:03:00:00']

def waitingTime3B():
    return ['waiting', '00:01:00:00', '00:15:00:00']

In [29]:
def build3A():
    night_duration = 480
    activities = []
    group = [nightSleep(), nightSleep(), nightSleep(), nightSleep(), nightSleep(), goBathroomNight()]
    activity = random.choice(group)
    current_state = activity
    while night_duration <= 480:
        if activity == nightSleep():
            if night_duration <= 60:
                break
            activity_duration = round(random.uniform(60,240),2)
        else:
            activity_duration = round(random.uniform(5, 15),2)
        activities.append([activity, activity_duration])
        night_duration -= activity_duration
        
        
        if activity == current_state:

            activity = random.choice(group)
        else:
            #print(activity, current_state)
            activities.insert(-1, [waitingTime3A(), round(random.uniform(1, 3),2)])
            current_state = activity
            activity = random.choice(group)
            
    activities.append([waitingTime3A(), round(random.uniform(1, 3),2)])
    return activities

In [30]:
def build3B():
    night_duration = 640
    activities = []
#     group = [nightSleep3B(), goBathroomNight3B(), nightSleep3B(), goBathroomNight3B(), nightSleep3B(), goBathroomNight3B(), wakeUp()]
    group = [nightSleep3B(), goBathroomNight3B(), nightSleep3B(), goBathroomNight3B(), nightSleep3B(), goBathroomNight3B()]#, wakeUp()]
    activity = random.choice(group)
    current_state = activity
    while night_duration <= 640:
        activity = random.choice(group)
        if activity == wakeUp():
            activities.append([activity, round(random.uniform(0,15),2)])
        if activity == nightSleep3B():
            if night_duration <= 60:
                break
            activity_duration = round(random.uniform(0,120),2)
        else:
            activity_duration = round(random.uniform(0,45),2)
        activities.append([activity, activity_duration])
        night_duration -= activity_duration
        
        
        if activity == current_state:
            activity = random.choice(group)
        else:
            #print(activity, current_state)
            activities.insert(-1, [waitingTime3A(), round(random.uniform(1, 10),2)])
            current_state = activity
            activity = random.choice(group)
            
    activities.append([waitingTime3A(), round(random.uniform(1, 10),2)])
    return activities

In [31]:
def mergeData(day):
    droprow = []
    result = day
    label_index = 0
    for i in range(1, day.shape[0]):
        if day.iloc[i]['concept:name'] == result.iloc[label_index]['concept:name']:
            result.iloc[label_index, 3] = day.iloc[i, 3]
            result.iloc[label_index, 4] += day.iloc[i, 4]
            droprow.append(i)
        else:
            label_index = i
    result = result.drop(index=droprow)
    result = result.reset_index()
    return result

In [32]:
dayActivities = build3A()
dayActivities.append([wakeUp(), round(random.uniform(1, 15),2)])
dd = pd.DataFrame(dayActivities)
dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)

In [33]:
def createEventLog3A(dayRange, fileName):
    path = './'+fileName.split('.')[0] 
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
        
    #xes_Index = ["case:concept:name","concept:name" , "start_time", "End_time", "Duration"]
    daySimulation = pd.DataFrame()
#     print(daySimulation)

#     Setting the start date (the first date in log)
    start_time = datetime(2022, 1, 1, 22, 0, 0, 0)
    
    for i in range(dayRange):
        # Create daily routine
        dayActivities = build3A()
        dayActivities.append([wakeUp(), round(random.uniform(1, 15),2)])
        dd = pd.DataFrame(dayActivities)
        dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)
        dd.columns = ['Activity', 'Min', 'Max', 'Duration']
        dd = dd.reindex(columns = ["Activity","Min", "Max", "start_time", "End_time", "Duration"]) 
        dd = dd.fillna("")

        # Randomly get time and calculate routine
#         start_time = datetime(2022, 1, i+1, 22, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S:%f'
#         Adding one day to the starting date
        start_time = start_time + timedelta(days=1)

        ddfinal = calculateDuration(dd, start_time)
        ddfinal = ddfinal.drop(columns =['Min', 'Max'])
        ddfinal = ddfinal.rename(columns={'Activity':'concept:name'})
        ddfinal = ddfinal.rename(columns={'End_time':'time:timestamp'})
        #ddfinal = ddfinal.rename(columns={'index':'case_id'})
        ddfinal['case_id'] = i+1
        # print(ddfinal)
        ddfinal.insert(loc=0, column='case:concept:name', value=i+1)
        #resfinal = resfinal.reindex(columns = ['case:concept:name', 'concept:name', 'Start_time', 'End_time', 'Duration'])
        ddfinal = mergeData(ddfinal)
        
#         print(ddfinal)
        ddfinal = ddfinal.drop(ddfinal[ddfinal['concept:name']=='waiting'].index)
    
        pm4py.write_xes(ddfinal, './%s/'%path + 'day%d.xes'%(i+1))
    #     Add daily routine to Final data
        daySimulation = daySimulation.append(ddfinal)
        
        # Create Event Log 3A

    #daySimulation = daySimulation.drop(daySimulation[daySimulation['concept:name']=='waiting'].index)
    #print(daySimulation)
    pm4py.write_xes(daySimulation, fileName)

In [34]:
def createEventLog3B(dayRange, fileName):
    path = './'+fileName.split('.')[0] 
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
    #xes_Index = ["case:concept:name","Activity" , "start_time", "End_time", "Duration"]
    daySimulation = pd.DataFrame()
    
#     Setting the start date (the first date in log)
    start_time = datetime(2022, 1, 1, 22, 0, 0, 0)

    for i in range(dayRange):
        # Create daily routine
        dayActivities = build3B()
        dayActivities.append([wakeUp(), round(random.uniform(1, 15),2)])
        dd = pd.DataFrame(dayActivities)
        dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)
        dd.columns = ['Activity', 'Min', 'Max', 'Duration']
        dd = dd.reindex(columns = ["Activity","Min", "Max", "start_time", "End_time", "Duration"]) 
        dd = dd.fillna("")
        #print(dd)
        # Randomly get time and calculate routine
#         start_time = datetime(2022, 1, i+1, 22, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S:%f')
#         Adding one day to the starting date
        start_time = start_time + timedelta(days=1)
    
        ddfinal = calculateDuration(dd, start_time)
        ddfinal = ddfinal.drop(columns =['Min', 'Max'])
        ddfinal = ddfinal.rename(columns={'Activity':'concept:name'})
        ddfinal = ddfinal.rename(columns={'End_time':'time:timestamp'})
        ddfinal.insert(loc=0, column='case:concept:name', value=i+1)
        ddfinal = mergeData(ddfinal)
        
        ddfinal = ddfinal.drop(ddfinal[ddfinal['concept:name']=='waiting'].index)
    
        pm4py.write_xes(ddfinal, './%s/'%path + 'day%d.xes'%(i+1))

    #     Add daily routine to Final data
        daySimulation = daySimulation.append(ddfinal)
        
        # Create Event Log 3A
    daySimulation = daySimulation.drop(daySimulation[daySimulation['concept:name']=='waiting'].index)
    pm4py.write_xes(daySimulation, fileName)

In [35]:
createEventLog3A(56, 'Night3A.xes')

createEventLog3A(7, 'Night3Ap.xes')
createEventLog3A(7, 'Night3Ap2.xes')

createEventLog3B(7, 'Night3B.xes')
createEventLog3B(7, 'Night3B2.xes')

exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_52329/692468135.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_52329/692468135.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

...

exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_52329/1767762639.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_52329/1767762639.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/7 [00:00<?, ?it/s]

# ------------------------------